In [1]:
using LinearAlgebra
using SymPy

# Override https://github.com/jverzani/SymPyCore.jl/blob/main/src/SymPy/show_sympy.jl#L31-L34
@eval SymPy begin
function Base.show(io::IO,  ::MIME"text/latex", x::SymbolicObject)
    out = _sympy_.latex(↓(x), mode="inline",fold_short_frac=false)
    out = replace(out, r"\\frac{"=>"\\dfrac{")
    print(io, string(out))
end
end

N = 3
@eval @syms u[1:$N] v[1:$N]
@show u v

b(i,j) = (u[i] - v[i])/(u[i] - v[j])
B(n) = [b(i,j) for i in 1:n, j in 1:n]
A(i,j) = (u[i] - u[j])*(v[i] - v[j])/((u[i] - v[j])*(v[i] - u[j]))
A(n) = prod(A(i, j) for i in 1:n for j in i+1:n)

u = Sym{PyCall.PyObject}[u₁, u₂, u₃]
v = Sym{PyCall.PyObject}[v₁, v₂, v₃]


A (generic function with 2 methods)

In [2]:
@time expr2 = det(B(2)) |> factor

  1.785509 seconds (275.92 k allocations: 18.938 MiB, 11.58% compilation time)


-(u₁ - u₂)*(v₁ - v₂) 
---------------------
 (u₁ - v₂)*(u₂ - v₁) 

In [3]:
expr2/A(2) |> simplify

1

In [4]:
@time expr3 = det(B(3)) |> factor

  3.988889 seconds (16.01 k allocations: 1.079 MiB, 0.32% compilation time)


-(u₁ - u₂)*(u₁ - u₃)*(u₂ - u₃)*(v₁ - v₂)*(v₁ - v₃)*(v₂ - v₃) 
-------------------------------------------------------------
 (u₁ - v₂)*(u₁ - v₃)*(u₂ - v₁)*(u₂ - v₃)*(u₃ - v₁)*(u₃ - v₂) 

In [5]:
expr3/A(3) |> simplify

1